### tflite custom operator

- https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/custom_operators.md

In [1]:
import tensorflow as tf
import numpy as np
import os

#### Model

- y = sin(x + 0.4)

In [2]:
offset = tf.get_variable("offset", [1,], tf.float32)
x = tf.placeholder(tf.float32, shape=(1,17))
y = tf.sin(x + offset)
y_ = tf.placeholder(tf.float32, shape=(1,17))
loss = tf.reduce_sum(tf.square(y - y_))
optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

#### Input data

In [3]:
np_x = np.array([-4.0, -3.5, -3.0, -2.5, -2.0, -1.5, -1.0, -0.5, 0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0], dtype=np.float32, ndmin=2)
np_y_ = np.array([0.4425, -0.0416, -0.5155, -0.8632, -0.9996, -0.8912, -0.5646, -0.0998, 0.3894, 0.7833, 0.9854, 
              0.9463, 0.6755, 0.2392, -0.2555, -0.6878, -0.9516], dtype=np.float32, ndmin=2)

#### Fit & convert to tflite model

- https://www.tensorflow.org/lite/convert/python_api

In [4]:
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(500):
        sess.run(train, feed_dict={x: np_x, y_: np_y_})
    
    print('\t loss =   %E' % sess.run(loss, feed_dict={x: np_x, y_: np_y_}))
    print('\t offset = %.4f' % sess.run(offset, feed_dict={x: np_x, y_: np_y_}))
        
    save_path = saver.save(sess, os.path.join(os.getcwd(), 'sin.ckpt'))
    
    converter = tf.contrib.lite.TFLiteConverter.from_session(sess, [x], [y])
    tflite_model = converter.convert()
    open("tflite_op_sine.tflite", "wb").write(tflite_model)

	 loss =   1.698754E-08
	 offset = 0.4000
INFO:tensorflow:Froze 1 variables.
INFO:tensorflow:Converted 1 variables to const ops.
